In [20]:
# Install ITK
import pip
pip.main(['install', '--upgrade', 'pip'])
pip.main(['install', 'itk', '-f', 'https://github.com/InsightSoftwareConsortium/ITKPythonPackage/releases/tag/nightly'])

import itk

Requirement already up-to-date: pip in /home/matt/bin/venvs/confidence-connected/lib/python2.7/site-packages


In [21]:
# Get a brain MRI dataset
pip.main(['install', 'requests'])
import requests

url = 'https://data.kitware.com/api/v1/file/588271308d777f4f3f3072e2/download'
filename = 'brainweb165a10f17.mha'

request = requests.get(url)
with open('brainweb165a10f17.mha', 'wb') as fp:
    fp.write(request.content)

In [25]:
# Read in the image
reader = itk.ImageFileReader.New(FileName=filename)
reader.Update()
input_image = reader.GetOutput()
print(input_image)

Image (0x3aec550)
  RTTI typeinfo:   itk::Image<unsigned char, 3u>
  Reference Count: 2
  Modified Time: 971
  Debug: Off
  Object Name: 
  Observers: 
    none
  Source: (0x641a850) 
  Source output name: Primary
  Release Data: Off
  Data Released: False
  Global Release Data: Off
  PipelineMTime: 814
  UpdateMTime: 972
  RealTimeStamp: 0 seconds 
  LargestPossibleRegion: 
    Dimension: 3
    Index: [0, 0, 0]
    Size: [181, 217, 180]
  BufferedRegion: 
    Dimension: 3
    Index: [0, 0, 0]
    Size: [181, 217, 180]
  RequestedRegion: 
    Dimension: 3
    Index: [0, 0, 0]
    Size: [181, 217, 180]
  Spacing: [1, 1, 1]
  Origin: [0, 0, 0]
  Direction: 
1 0 0
0 1 0
0 0 1

  IndexToPointMatrix: 
1 0 0
0 1 0
0 0 1

  PointToIndexMatrix: 
1 0 0
0 1 0
0 0 1

  Inverse Direction: 
1 0 0
0 1 0
0 0 1

  PixelContainer: 
    ImportImageContainer (0x3310ec0)
      RTTI typeinfo:   itk::ImportImageContainer<unsigned long, unsigned char>
      Reference Count: 1
      Modified Time: 969
      D

In [26]:
# The image type we will use to process the image
PixelType = itk.ctype('float')
Dimension = 3

ImageType = itk.Image[PixelType, Dimension]

In [28]:
InputImageType = type(input_image)
caster = itk.CastImageFilter[InputImageType, ImageType].New(input_image)

In [48]:
# Smooth the image
smoother = itk.CurvatureFlowImageFilter.New(caster.GetOutput())
smoother.SetNumberOfIterations(6)
smoother.SetTimeStep(0.05)

In [49]:
confidence_connected = itk.ConfidenceConnectedImageFilter[ImageType, InputImageType].New()
confidence_connected.SetInput(smoother.GetOutput())
confidence_connected.SetMultiplier(2.5)
confidence_connected.SetNumberOfIterations(5)
confidence_connected.SetInitialNeighborhoodRadius(2)
confidence_connected.SetReplaceValue(255)

SeedType = itk.Index[Dimension]
seed1 = SeedType()
seed1[0] = 118
seed1[1] = 133
seed1[2] = 92
confidence_connected.AddSeed(seed1)

seed2 = SeedType()
seed2[0] = 63
seed2[1] = 135
seed2[2] = 94
confidence_connected.AddSeed(seed2)

seed3 = SeedType()
seed3[0] = 63
seed3[1] = 157
seed3[2] = 90
confidence_connected.AddSeed(seed3)

seed4 = SeedType()
seed4[0] = 111
seed4[1] = 150
seed4[2] = 90
confidence_connected.AddSeed(seed4)

seed5 = SeedType()
seed5[0] = 111
seed5[1] = 50
seed5[2] = 88
confidence_connected.AddSeed(seed5)

In [50]:
writer = itk.ImageFileWriter.New(confidence_connected.GetOutput())
writer.SetFileName('WhiteMatterSegmentation.mha')

In [51]:
# Runs the processing pipeline
writer.Update()